In [7]:
from pyproj import Transformer
import pandas as pd

# Latitude and longitude
#latitude = 40.1428149996863
#longitude = -7.37379500018226

NaturalFires = pd.read_csv('NaturalFires.csv')


with rasterio.open('u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif') as src:
    # Create a transformer from WGS 84 (EPSG:4326) to the CRS of the raster data
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3035")

    

    for i in range(len(NaturalFires)):
        latitude = NaturalFires['latitude'][i]
        longitude = NaturalFires['longitude'][i]

        # Transform the coordinates
        longitude_transformed, latitude_transformed = transformer.transform(latitude, longitude)

        # Get the row and column indices
        col, row = src.index(longitude_transformed, latitude_transformed)

        print('Transformed coordinates:', (longitude_transformed, latitude_transformed))
        print('Raster bounds:', src.bounds)

        #print(src.meta)
        #print(src.transform)
        #print(src.bounds)
        #print(src.crs)
        #print('Width:', src.width)
        #print('Height:', src.height)
        #print('No-data value:', src.nodata)
        
        # Check if the row and col indices are within the bounds of the raster data array
        if 0 <= row < src.height and 0 <= col < src.width:
            # Sample the raster at the given coordinates
            value = src.read(1)[row, col]
            clc_code = value.item()

            # Check if the value is a no-data value
            if clc_code in src.nodatavals:
                print('No data at this location')
            else:
                print('CLC code:', clc_code)
        else:
            print('The provided coordinates are outside the bounds of the raster data.')

Transformed coordinates: (1779265.4280139445, 2678563.5081761396)
Raster bounds: BoundingBox(left=900000.0, bottom=900000.0, right=7400000.0, top=5500000.0)
No data at this location
Transformed coordinates: (1774639.6980806661, 2672974.1447090497)
Raster bounds: BoundingBox(left=900000.0, bottom=900000.0, right=7400000.0, top=5500000.0)
No data at this location
Transformed coordinates: (2106591.6854123445, 2844583.6496685063)
Raster bounds: BoundingBox(left=900000.0, bottom=900000.0, right=7400000.0, top=5500000.0)
No data at this location
Transformed coordinates: (2103691.7107221493, 2836479.0610621828)
Raster bounds: BoundingBox(left=900000.0, bottom=900000.0, right=7400000.0, top=5500000.0)
No data at this location
Transformed coordinates: (2086322.4915779312, 2826514.577784892)
Raster bounds: BoundingBox(left=900000.0, bottom=900000.0, right=7400000.0, top=5500000.0)
No data at this location
Transformed coordinates: (2093556.0896140952, 2822455.3306278745)
Raster bounds: BoundingBo

KeyboardInterrupt: 

In [10]:
from osgeo import gdal, osr
import pandas as pd
import numpy as np

# Load the CSV file
NaturalFires = pd.read_csv('NaturalFires.csv')

# Open the raster file
ds = gdal.Open('u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif')

# Get the raster band
band = ds.GetRasterBand(1)

# Get the NoData value
nodata = band.GetNoDataValue()

# Get the GeoTransform
gt = ds.GetGeoTransform()

# Create a spatial transformer
src_srs = osr.SpatialReference()
src_srs.ImportFromEPSG(3857)  # Web Mercator
tgt_srs = osr.SpatialReference()
tgt_srs.ImportFromWkt(ds.GetProjectionRef())  # The CRS of the raster data
transform = osr.CoordinateTransformation(src_srs, tgt_srs)

print(ds.GetProjection())

for i in range(len(NaturalFires)):
    latitude = NaturalFires['latitude'][i]
    longitude = NaturalFires['longitude'][i]

    # Transform the coordinates
    point = transform.TransformPoint(longitude, latitude)

    # Get the row and column indices
    col = int((point[0] - gt[0]) / gt[1])
    row = int((point[1] - gt[3]) / gt[5])

    # Check if the row and col indices are within the bounds of the raster data array
    if 0 <= row < ds.RasterYSize and 0 <= col < ds.RasterXSize:
        # Sample the raster at the given coordinates
        value = band.ReadAsArray(col, row, 1, 1)

        # Check if the value is a no-data value
        if value == nodata:
            print('No data at this location')
        else:
            print('CLC code:', value[0, 0])
    else:
        print('The provided coordinates are outside the bounds of the raster data.')

PROJCS["ETRS89-extended / LAEA Europe",GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4258"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",52],PARAMETER["longitude_of_center",10],PARAMETER["false_easting",4321000],PARAMETER["false_northing",3210000],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3035"]]
The provided coordinates are outside the bounds of the raster data.
The provided coordinates are outside the bounds of the raster data.
The provided coordinates are outside the bounds of the raster data.
The provided coordinates are outside the bounds of the raster data.
The provided coordinates are outside the bounds of the raster data.
The provided coordinates are outside the bounds of the raster data.
The provide

In [6]:
def test_transform_coordinates():
    # Expected transformed coordinates
    expected_longitude_transformed = 2063966.0997659448
    expected_latitude_transformed = 2847504.7457216308

    # Latitude and longitude
    latitude = 40.1428149996863
    longitude = -7.37379500018226

    with rasterio.open('u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif') as src:
        # Create a transformer from WGS 84 (EPSG:4326) to the CRS of the raster data
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:3035")

        # Transform the coordinates
        longitude_transformed, latitude_transformed = transformer.transform(latitude, longitude)

        # Assert that the transformed coordinates are as expected
        assert longitude_transformed == expected_longitude_transformed
        assert latitude_transformed == expected_latitude_transformed

test_transform_coordinates()

In [ ]:
import rasterio
import numpy as np

with rasterio.open('u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif') as src:
    # Initialize an empty set to store the unique CLC codes
    unique_clc_codes = set()

    # Process the raster data in chunks
    for ji, window in src.block_windows(1):
        # Read a chunk of the raster data
        data = src.read(1, window=window)

        # Add the unique values in this chunk to the set of unique CLC codes
        unique_clc_codes.update(np.unique(data))

    # Print the unique CLC codes
    print('Unique CLC codes:', unique_clc_codes)

: 